#Descifrar RSA con computación cuántica

#Integrantes

-Arana Flores Shaiel Almendra

-Huerta Medina Vito Jhon

-Lima Esperilla Katerine Candy

-Mamani Meza Johana Maria

-Poma Supo Juan Gabriel

#Instalación

In [ ]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.7 MB/s eta 0:00:00


In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.6 MB/s eta 0:00:00


#Modulos Requeridos

In [ ]:
from __future__ import unicode_literals
from math import sqrt
import random
from random import randint as rand

# Función para calcular el máximo común divisor (GCD) de dos números
def calcular_mcd(a, b):
    if b == 0:
        return a
    else:
        return calcular_mcd(b, a % b)

# Función para calcular el inverso modular
def inverso_modular(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return -1

# Función para verificar si un número es primo
def es_primo(n):
    if n < 2:
        return False
    elif n == 2:
        return True
    else:
        for i in range(1, int(sqrt(n)) + 1):
            if n % i == 0:
                return False
    return True
# Función para verificar si un número es primo
def es_primo(n):
    # Un número menor que 2 no es primo
    if n < 2:
        return False
    # El número 2 es primo
    elif n == 2:
        return True
    else:
        # Verificar si n es divisible por cualquier número desde 2 hasta la raíz cuadrada de n
        for i in range(2, int(sqrt(n)) + 1):
            if n % i == 0:
                return False
    return True

# Función para generar un par de claves pública y privada
def generar_claves(tamano_clave):
    # Generar dos números aleatorios entre 1 y 1000
    p = rand(1, 1000)
    q = rand(1, 1000)

    # Calcular el valor mínimo y máximo de n para el tamaño de clave deseado
    n_min = 1 << (tamano_clave - 1)  # 2^(tamano_clave - 1)
    n_max = (1 << tamano_clave) - 1  # 2^tamano_clave - 1

    # Lista inicial de números primos con el primer primo conocido
    primos = [2]

    # Calcular los límites de inicio y fin para la búsqueda de primos
    inicio = 1 << (tamano_clave // 2 - 1)
    fin = 1 << (tamano_clave // 2 + 1)

    # Si el inicio es mayor o igual que el fin, no se pueden generar claves válidas
    if inicio >= fin:
        return []

    # Generar lista de números primos hasta el límite de fin
    for i in range(3, fin + 1, 2):
        for p in primos:
            if i % p == 0:
                break
        else:
            primos.append(i)

    # Eliminar números primos menores que el inicio
    while (primos and primos[0] < inicio):
        del primos[0]

    # Seleccionar números primos p y q dentro del rango deseado
    while primos:
        p = random.choice(primos)
        primos.remove(p)

        # Filtrar valores posibles de q que cumplen con la condición n_min <= p * q <= n_max
        q_valores = [q for q in primos if n_min <= p * q <= n_max]
        if q_valores:
            q = random.choice(q_valores)
            break

    # Imprimir los primos seleccionados p y q
    print(p, q)

    # Calcular n y phi (totient de Euler)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Seleccionar un número e aleatorio tal que 1 < e < phi y gcd(e, phi) = 1
    e = random.randrange(1, phi)
    g = calcular_mcd(e, phi)

    # Repetir hasta encontrar un e coprimo con phi
    while True:
        e = random.randrange(1, phi)
        g = calcular_mcd(e, phi)
        d = inverso_modular(e, phi)
        if g == 1 and e != d:
            break

    # Retornar las claves pública (e, n) y privada (d, n)
    return ((e, n), (d, n))


# Función para cifrar un mensaje de texto
def cifrar(mensaje_texto, paquete):
    e, n = paquete
    mensaje_cifrado = [pow(ord(c), e, n) for c in mensaje_texto]
    return ''.join(map(lambda x: str(x), mensaje_cifrado)), mensaje_cifrado

# Función para descifrar un mensaje cifrado
def descifrar(mensaje_cifrado, paquete):
    d, n = paquete
    mensaje_descifrado = [chr(pow(c, d, n)) for c in mensaje_cifrado]
    return (''.join(mensaje_descifrado))


In [ ]:
# Solicita al usuario que ingrese el tamaño en bits para la generación de claves
bit_length = int(input("Ingrese el tamaño en bits: "))

# Genera un par de claves (pública y privada) basado en el tamaño en bits ingresado
publica, privada = generar_claves(2**bit_length)


56501 40813


#RSA Encriptación

In [ ]:
# Solicita al usuario que escriba un mensaje para cifrar
mensaje = input("\nEscriba el mensaje: ")

# Cifra el mensaje utilizando la clave pública generada
mensaje_cifrado, objeto_cifrado = cifrar(mensaje, publica)

# Imprime el mensaje cifrado
print("\nMensaje cifrado: " + mensaje_cifrado)



Escriba el mensaje: hola

Mensaje cifrado: 5632121236431541208


#RSA Descifrar

In [ ]:
# Descifra el mensaje cifrado utilizando la clave privada
mensaje_descifrado = descifrar(objeto_cifrado, privada)

# Imprime el mensaje descifrado
print("\nMensaje descifrado utilizando el Algoritmo RSA: " + mensaje_descifrado)



Mensaje descifrado utilizando el Algoritmo RSA: hola


#Algoritmo cuántico de Shor

In [ ]:
!pip install qiskit-aer
from math import gcd,log
from random import randint
import numpy as np
from qiskit import *
from qiskit_aer import AerSimulator


# Use qiskit.Aer to get the backend
qasm_sim = Aer.get_backend('qasm_simulator')


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import Aer
import numpy as np
from random import randint

# Obtén el backend del simulador de QASM
qasm_sim = Aer.get_backend('qasm_simulator')

def periodo(a, N):
    available_qubits = 16  # Número de qubits disponibles en el simulador
    r = -1  # Inicializa r como -1 para usarlo como contador

    # Verifica si N es demasiado grande para el simulador
    if N >= 2**available_qubits:
        print(f"{N} es demasiado grande para el simulador")
        return r

    # Crea registros cuánticos y clásicos
    qr = QuantumRegister(available_qubits)
    cr = ClassicalRegister(available_qubits)
    qc = QuantumCircuit(qr, cr)

    # Inicializa x0 y su representación binaria
    x0 = randint(1, N - 1)  # Escoge un x0 aleatorio entre 1 y N-1
    x_binario = np.zeros(available_qubits, dtype=bool)  # Inicializa x_binario como un array de ceros

    # Convierte N a binario y lo almacena en x_binario
    for i in range(1, available_qubits + 1):
        bit_state = (N % (2**i) != 0)
        if bit_state:
            N -= 2**(i - 1)
        x_binario[available_qubits - i] = bit_state

    # Configura el estado inicial del circuito cuántico
    for i in range(available_qubits):
        if x_binario[available_qubits - i - 1]:
            qc.x(qr[i])  # Aplica la puerta X a los qubits que corresponden a 1 en x_binario

    x = x0
    while x != x0 or r <= 0:
        r += 1  # Incrementa el contador r

        # Mide el circuito cuántico
        qc.measure(qr, cr)

        # Define las puertas cuánticas (esta sección necesita lógica específica según el algoritmo)
        for i in range(0, 3):
            qc.x(qr[i])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[2])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[3], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])

        # Transpila el circuito para el backend
        transpiled_circuit = transpile(qc, backend=qasm_sim)

        # Compila el circuito transpileado en un formato ejecutable
        qobj = assemble(transpiled_circuit, shots=1024)

        # Ejecuta el circuito
        resultado = qasm_sim.run(qobj).result()
        counts = resultado.get_counts()

        # Encuentra el resultado más frecuente
        resultados = [[], []]
        for key, value in counts.items():
            resultados[0].append(key)
            resultados[1].append(int(value))
        s = resultados[0][np.argmax(np.array(resultados[1]))]

        # Condición dummy para salir del bucle (añadir lógica real)
        if len(counts) > 0:
            break

    return r  # Retorna el período encontrado


In [ ]:
def rompe_shor(N):
    N = int(N)  # Convierte N a un entero
    while True:
        a = randint(0, N - 1)  # Genera un número aleatorio a en el rango [0, N-1]
        g = gcd(a, N)  # Calcula el máximo común divisor de a y N
        if g != 1 or N == 1:
            return g, N // g  # Si g no es 1 o N es 1, retorna g y N dividido por g (factor trivial)
        else:
            r = periodo(a, N)  # Calcula el periodo r de la función a^x mod N
            if r % 2 != 0:
                continue  # Si r es impar, continúa con la siguiente iteración del bucle
            elif pow(a, r // 2, N) == -1:
                continue  # Si a^(r/2) mod N es -1, continúa con la siguiente iteración del bucle
            else:
                p = gcd(pow(a, r // 2) + 1, N)  # Calcula el máximo común divisor de (a^(r/2) + 1) y N
                q = gcd(pow(a, r // 2) - 1, N)  # Calcula el máximo común divisor de (a^(r/2) - 1) y N
                if p == N or q == N:
                    continue  # Si p o q son iguales a N, continúa con la siguiente iteración del bucle
                return p, q  # Retorna los factores no triviales p y q de N


In [ ]:
def inverso_modular(a,m):
    a = a % m;
    for x in range(1, m) :
        if ((a * x) % m == 1) :
            return x
    return 1


In [ ]:
# Obtiene el valor de N a partir de la clave pública
N_shor = public[1]

# Verifica que N_shor sea positivo
assert N_shor > 0, "La entrada debe ser un número positivo"

# Llama a la función shors_breaker para encontrar los factores p y q de N_shor
p, q = shors_breaker(N_shor)

# Calcula la función totiente de Euler para el producto de p y q
phi = (p - 1) * (q - 1)

# Calcula el inverso modular de la clave pública e con respecto a phi
d_shor = inverso_modular(public[0], phi)


<ipython-input-67-3168c754cbdf>:60: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = qasm_sim.run(qobj).result()


#Descifrando RSA usando el algoritmo de Shor

In [ ]:
mensaje_descifrado = decrypt(encryption_obj, (d_shor, N_shor))

print('\nMensaje descifrado usando el algoritmo de Shor: ' + mensaje_descifrado + "\n")



Message Cracked using Shors Algorithm: hola



# Test - con 29 bits

In [ ]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
#!pip install qiskit #==0.14.1 #qiskit-aer==0.3.2 numpy==1.16.2
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.7 MB/s eta 0:00:00


In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.9 MB/s eta 0:00:00


In [ ]:
from __future__ import unicode_literals
from math import sqrt
import random
from random import randint as rand

# Función para calcular el máximo común divisor
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)

# Función para calcular el inverso modular
def inverso_modular(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return -1

# Función para verificar si un número es primo
def es_primo(n):
    if n < 2:
        return False
    elif n == 2:
        return True
    else:
        for i in range(2, int(sqrt(n)) + 1):
            if n % i == 0:
                return False
    return True

# Función para generar un par de claves (pública y privada)
def generar_par_claves(tamaño_clave):
    p = rand(1, 1000)
    q = rand(1, 1000)
    nMin = 1 << (tamaño_clave - 1)
    nMax = (1 << tamaño_clave) - 1
    primos = [2]
    inicio = 1 << (tamaño_clave // 2 - 1)
    fin = 1 << (tamaño_clave // 2 + 1)
    if inicio >= fin:
        return []
    for i in range(3, fin + 1, 2):
        for p in primos:
            if i % p == 0:
                break
        else:
            primos.append(i)
    while (primos and primos[0] < inicio):
        del primos[0]
    while primos:
        p = random.choice(primos)
        primos.remove(p)
        valores_q = [q for q in primos if nMin <= p * q <= nMax]
        if valores_q:
            q = random.choice(valores_q)
            break
    print(p, q)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = random.randrange(1, phi)
    g = gcd(e, phi)
    while True:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
        d = inverso_modular(e, phi)
        if g == 1 and e != d:
            break

    return ((e, n), (d, n))

# Función para encriptar un mensaje
def encriptar(msg_texto_plano, paquete):
    e, n = paquete
    msg_texto_cifrado = [pow(ord(c), e, n) for c in msg_texto_plano]
    return ''.join(map(lambda x: str(x), msg_texto_cifrado)), msg_texto_cifrado

# Función para desencriptar un mensaje
def desencriptar(msg_texto_cifrado, paquete):
    d, n = paquete
    msg_texto_plano = [chr(pow(c, d, n)) for c in msg_texto_cifrado]
    return ''.join(msg_texto_plano)


In [ ]:
# Comentado para la entrada del usuario y uso del valor fijo para el tamaño de la clave
# bit_length = int(input("Enter bit_length: "))

# Genera un par de claves públicas y privadas con un tamaño de clave de 29 bits
public, private = generar_par_claves(29)


###RSA Encriptacion

In [ ]:
#msg = input("\nWrite message: ")
msg = "Hola mundo"

# Encriptar el mensaje usando la clave pública
encrypted_msg, encryption_obj = encriptar(msg, public)

# Imprimir el mensaje cifrado
print("\nMensaje cifrado: " + encrypted_msg)

# Imprimir el objeto de cifrado (lista de valores cifrados)
print("\nObjeto de cifrado: " + str(encryption_obj))
print(encryption_obj)



Mensaje cifrado: 345890380319760064126640379305559700214494704189563492257053956235126621109022694319760064

Objeto de cifrado: [345890380, 319760064, 126640379, 305559700, 214494704, 189563492, 257053956, 235126621, 109022694, 319760064]
[345890380, 319760064, 126640379, 305559700, 214494704, 189563492, 257053956, 235126621, 109022694, 319760064]


###RSA Desencriptar

In [ ]:
# Desencriptar el mensaje usando la clave privada
mensaje_descifrado = desencriptar(encryption_obj, private)

# Imprimir el mensaje descifrado usando el algoritmo RSA
print("\nMensaje descifrado usando el algoritmo RSA: " + mensaje_descifrado)


Mensaje descifrado usando el algoritmo RSA: Hola mundo


### Algoritmo de Short

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import Aer
import numpy as np
from random import randint

# Obtén el backend del simulador de QASM
qasm_sim = Aer.get_backend('qasm_simulator')

def period(a, N):
    available_qubits = 29
    r = -1

    if N >= 2**available_qubits:
        print(f"{N} is too big for IBMQX")
        return r

    qr = QuantumRegister(available_qubits)
    cr = ClassicalRegister(available_qubits)
    qc = QuantumCircuit(qr, cr)

    x0 = randint(1, N - 1)
    x_binary = np.zeros(available_qubits, dtype=bool)

    for i in range(1, available_qubits + 1):
        bit_state = (N % (2**i) != 0)
        if bit_state:
            N -= 2**(i - 1)
        x_binary[available_qubits - i] = bit_state

    for i in range(available_qubits):
        if x_binary[available_qubits - i - 1]:
            qc.x(qr[i])

    x = x0
    while x != x0 or r <= 0:
        r += 1

        qc.measure(qr, cr)

        # Define las puertas cuánticas
        for i in range(0, 3):
            qc.x(qr[i])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[2])
        qc.cx(qr[2], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])
        qc.cx(qr[3], qr[0])
        qc.cx(qr[0], qr[1])
        qc.cx(qr[1], qr[0])

        # Transpila el circuito para el backend
        transpiled_circuit = transpile(qc, backend=qasm_sim)

        # Compila el circuito transpileado en un formato ejecutable
        qobj = assemble(transpiled_circuit, shots=1024)

        # Ejecuta el circuito
        result = qasm_sim.run(qobj).result()
        counts = result.get_counts()

        # Encuentra el resultado más frecuente
        results = [[], []]
        for key, value in counts.items():
            results[0].append(key)
            results[1].append(int(value))
        s = results[0][np.argmax(np.array(results[1]))]

        # Condición dummy para salir del bucle (añadir lógica real)
        if len(counts) > 0:
            break

    return r

def shors_breaker(N):
    N = int(N)
    while True:
        a=randint(0,N-1)
        g=gcd(a,N)
        if g!=1 or N==1:
            return g,N//g
        else:
            r=period(a,N)
            if r % 2 != 0:
                continue
            elif pow(a,r//2,N)==-1:
                continue
            else:
                p=gcd(pow(a,r//2)+1,N)
                q=gcd(pow(a,r//2)-1,N)
                if p==N or q==N:
                    continue
                return p,q

def modular_inverse(a,m):
    a = a % m;
    for x in range(1, m) :
        if ((a * x) % m == 1) :
            return x
    return 1

N_shor = public[1]
assert N_shor>0,"Input must be positive"
p,q = shors_breaker(N_shor)
phi = (p-1) * (q-1)
d_shor = modular_inverse(public[0], phi)

decrypted_msg = decrypt(encryption_obj, (d_shor,N_shor))

print('\nMensaje descifrado usando el algoritmo de Shor: ' + decrypted_msg + "\n")

<ipython-input-8-639f2f67ba4f>:60: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = qasm_sim.run(qobj).result()



Mensaje descifrado usando el algoritmo de Shor: Hola mundo

